# Introduction
Introduce the problem you are trying to solve, questions you are trying to answer, and/or overall goal of the project.

# About the Data
Describe where you got your data. This could include obtaining the data through Kaggle or other resources, or scraping the data yourself somehow.   
   
Also provide information about the dataset itself (what features, the size of the dataset, and any additional statistics or visualizations you create to better understand the data).

# Methods
This should include descriptions of any pre-processing steps and the modeling. If you experiment by trying things out in different iterations, it is encouraged to also talk about all of that! What worked or what didn’t? Were you able to improve upon the model(s) or gain new insights through different iterations?

# Evaluation
Based on your goal/model — how do you evaluate performance? What are the results?

For example, if you are running classification models, what are the accuracies, f1 scores, etc.? If you had questions in the introduction, were you able to answer them (discuss that)? etc.

Think about the best way to evaluate based on what you are trying to accomplish.

# Storytelling and Conclusion
What insights did you gain through your project? Were you able to answer your initial problems? Obtain your initial goal? What stories can you tell? Do you have future steps in mind or things that could have been improved? ***For full credit here, you will need to show your critical thinking throughout.***

What have you learned throughout this project and the entire class?

# Impact Section
Discuss the impact of your project. This can be socially, ethically, etc. It cannot be something like "our project has no impact" or "our project has no negative impact." Even the most well-intentioned projects *could* have negative impact. We will not be checking for "right" or "wrong" answers, but showing your critical thinking.

# Github Repository/Code/Data
Make sure to include the link to the repository and we are able to view it. Also include access to any data used.

https://github.com/matthewasloan95/chess